<a href="https://colab.research.google.com/github/SrijaniBasu/GenerativeAI_EmotionTrainerApp/blob/main/EmotionTrainer_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install openai
!pip install diffusers transformers accelerate


In [9]:
!pip install streamlit openai pyngrok

In [10]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [11]:
!pip install streamlit pyngrok openai -q

import os
from pyngrok import ngrok
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
ngrok_token = userdata.get("NGORK_TOKEN")
!ngrok config add-authtoken "$ngrok_token"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
%%writefile visual_empathy_app.py
import streamlit as st
from openai import OpenAI
import json
import re
import base64
from io import BytesIO

# Initialize OpenAI client (reads OPENAI_API_KEY from environment)
client = OpenAI()

# ---------- Image prompt helpers ----------
EMOTION_PROMPTS = {
    "happy":   "warm golden hour lighting, smiling subject, vibrant colors, cinematic portrait",
    "sad":     "cool blue tones, rain on window, somber mood, soft shadows, cinematic portrait",
    "angry":   "dramatic red accents, harsh contrast, tense expression, stormy sky, cinematic portrait",
    "fear":    "low-key lighting, subtle fog, eerie atmosphere, muted colors, cinematic portrait",
    "surprise":"bright highlights, widened eyes, dynamic composition, cinematic portrait",
    "disgust": "greenish tint, wrinkled nose expression, messy textures, close-up, cinematic portrait",
    "neutral": "balanced lighting, natural tones, relaxed expression, documentary portrait"
}

def build_image_prompt(emotion: str, scenario: str) -> str:
    """Build a diffusion-style prompt using the scenario text + emotion mood."""
    mood = EMOTION_PROMPTS.get(emotion.lower().strip(), EMOTION_PROMPTS["neutral"])
    style = "photorealistic, ultra-detailed, 50mm lens"
    return f"{scenario.strip()}, {mood}, {style}"

# ---------- Streamlit page config ----------
st.set_page_config(page_title="Visual Empathy Trainer", page_icon="🧠")
st.title("🧠 Visual Empathy Trainer")
st.subheader("Practice empathy in realistic social situations")

# Emotion selector
emotion = st.selectbox(
    "Choose the main emotion in the scene:",
    ["happy", "sad", "angry", "fear", "surprise", "disgust", "neutral"],
    index=1
)

# Session state initialization
if "scenario" not in st.session_state:
    st.session_state["scenario"] = ""
if "scenario_image" not in st.session_state:
    # ('type', value) where type is 'url' or 'bytes'
    st.session_state["scenario_image"] = None

# ---------- Step 1: Generate scenario + image ----------
st.markdown("### Step 1 — Generate a scenario and image")

if st.button("Generate scenario + image"):
    # 1) Generate scenario with GPT
    with st.spinner("🧠 Generating scenario..."):
        scenario_prompt = (
            f"Generate a short, realistic social scenario (3–6 sentences) where "
            f"one person feels {emotion}. Do NOT write bullet points, just plain text."
        )
        chat_resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an empathy training assistant."},
                {"role": "user", "content": scenario_prompt}
            ]
        )
        scenario_text = chat_resp.choices[0].message.content.strip()
        st.session_state["scenario"] = scenario_text

    # 2) Generate image from the scenario text
    with st.spinner("🎨 Generating image from the scenario..."):
        img_prompt = build_image_prompt(emotion, scenario_text)
        try:
            img_resp = client.images.generate(
                model="gpt-image-1",
                prompt=img_prompt,
                size="1024x1024"   # valid size
            )

            img_data = img_resp.data[0]

            # Try URL first (some environments use this)
            image_url = getattr(img_data, "url", None)
            b64_json = getattr(img_data, "b64_json", None)

            if image_url:
                st.session_state["scenario_image"] = ("url", image_url)
            elif b64_json:
                # Decode base64 into bytes
                img_bytes = base64.b64decode(b64_json)
                st.session_state["scenario_image"] = ("bytes", img_bytes)
            else:
                st.warning("⚠️ Image was generated but no usable data was returned.")
                st.session_state["scenario_image"] = None

        except Exception as e:
            st.warning(f"⚠️ Could not generate image: {e}")
            st.session_state["scenario_image"] = None

    st.success("✅ Scenario and image ready!")

# ---------- Step 2: Show scenario + image & collect reply ----------
if st.session_state["scenario"]:
    st.markdown("### 🧩 Scenario")
    st.write(st.session_state["scenario"])

    img_info = st.session_state.get("scenario_image", None)
    if img_info is not None:
        kind, value = img_info
        if kind == "url":
            st.image(
                value,
                caption=f"Generated scene for emotion: {emotion}",
                use_container_width=True
            )
        elif kind == "bytes":
            st.image(
                value,
                caption=f"Generated scene for emotion: {emotion}",
                use_container_width=True
            )
    else:
        st.info("No image available for this scenario yet.")

    st.markdown("### Step 2 — Your response")
    user_reply = st.text_area(
        "How would you respond to this situation?",
        placeholder="Write your message to the other person here..."
    )

    # ---------- Step 3: Get feedback ----------
    if st.button("Get Feedback"):
        if not user_reply.strip():
            st.error("Please write a response first.")
        else:
            with st.spinner("💬 Analyzing your response..."):
                feedback_prompt = f"""
                You are an empathy coach. The user responded: "{user_reply}"
                to the following scenario: "{st.session_state['scenario']}".
                Evaluate empathy in terms of awareness, validation, and tone (0–1 scale).
                Then give short constructive feedback.
                Respond ONLY as valid JSON with fields:
                awareness (float), validation (float), tone (float), feedback (string).
                """

                feedback_response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {
                            "role": "system",
                            "content": "Empathy feedback evaluator. Respond only with valid JSON — no markdown formatting."
                        },
                        {"role": "user", "content": feedback_prompt}
                    ]
                )
                raw_output = feedback_response.choices[0].message.content

                # Clean accidental ```json wrappers
                cleaned = re.sub(r"```(json)?", "", raw_output).strip().strip("`")

                try:
                    data = json.loads(cleaned)
                    st.markdown("### 🧠 Feedback Results")
                    st.json(data)
                except Exception:
                    st.error("⚠️ Could not parse the model output as JSON. Showing raw text below:")
                    st.write(raw_output)


Overwriting visual_empathy_app.py


In [13]:
# Run the Streamlit app
!streamlit run visual_empathy_app.py &>/dev/null &

# Create public URL with ngrok
from pyngrok import ngrok

# Optional: kill old tunnels if you re-run
ngrok.kill()

public_url = ngrok.connect(8501)
print("Your app is running at:", public_url)


Your app is running at: NgrokTunnel: "https://perthitically-medicamentous-hipolito.ngrok-free.dev" -> "http://localhost:8501"
